In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
import re

#----------스크롤링 코드------------------
driver = webdriver.Chrome('./chromedriver.exe')
url='https://www.facebook.com/'
driver.get(url)

In [2]:
SCROLL_PAUSE_TIME=2
body = driver.find_element_by_tag_name('body')
while True:
    last_height=driver.execute_script('return document.documentElement.scrollHeight')
    for i in range(10):
        body.send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
        soup = bs(driver.page_source, 'html.parser')
    new_height= driver.execute_script('return document.documentElement.scrollHeight')
    if new_height == last_height:
        break

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=87.0.4280.88)


In [3]:
content=soup.select('.du4w35lb.k4urcfbm.l9j0dhe7.sjgh65i0')
content_str=[]
for i in range(len(content)):
    content_str.append(str(content[i]))
#내용   
content_list=[]
for i in range(len(content)):
    content_list.append(content[i].get_text())
content_list2=[]
for i in range(len(content_list)):
    if '댓글' in content_list[i] and '공유' in content_list[i]:
        detgul=content_list[i].index('댓글')
        gongyu=content_list[i].index('공유')
        if detgul<gongyu:
            content_list2.append(content_list[i][0:detgul])
        else:
            content_list2.append(content_list[i][0:gongyu])
    elif '댓글' in content_list[i] and '공유' not in content_list[i]:
        detgul=content_list[i].index('댓글')
        content_list2.append(content_list[i][0:detgul])
#게시일
time=soup.select('.pybr56ya.dati1w0a.hv4rvrfc.n851cfcs.btwxx1t3.j83agx80.ll8tlv6m')
time_list=[]
for i in range(len(time)):
    time_list.append(time[i].get_text())
time_list2=[]
for i in range(len(time)):
    temp1=re.sub('[=]','',time_list[i])
    time_list2.append(temp1)
time_list3=[]
for i in range(len(time)):
    temp2=re.sub('[a-z+]','',time_list2[i])
    time_list3.append(temp2)
time_list4=[]
for i in range(len(time)):
    temp3=re.sub('[A-Z+]','',time_list3[i])
    time_list4.append(temp3)
time_list5=[]
for i in range(len(time)):
    temp4=re.sub('[\xa0]','',time_list4[i])
    time_list5.append(temp4)
time_list6=[]
for i in range(len(time)):
    temp5=re.sub('[^\d+월 \d+일]','',time_list5[i])
    time_list6.append(temp5.strip())
#반응 수
reaction_list=[]
for i in range(len(content_str)):
    if 'gpro0wi8 pcp91wgn' not in content_str[i]:
        reaction_list.append(0)
    else:
        abc=content_str[i].index('gpro0wi8 pcp91wgn')
        reaction=content_str[i][abc:]
        de=reaction.index('<')
        reaction_list.append(reaction[19:de])
#댓글, 공유 수        
comment_gongyu_list=[]
for i in range(len(content_str)):
    if 'bp9cbjyn j83agx80 pfnyh3mw p1ueia1e' in content_str[i]:
        if content_str[i].count('gtad4xkn')==0:
            comment_gongyu_list.append('댓글 0개, 공유 0회')
        elif content_str[i].count('gtad4xkn')==1:
            share=re.findall('공유 \d....',content_str[i])
            comment=re.findall('댓글 \d....',content_str[i])
            if len(share)!=0:
                realshare='댓글 0개, '+share[0]
                comment_gongyu_list.append(realshare)
            else:
                realcomment=comment[0]+', 공유 0회'
                comment_gongyu_list.append(realcomment)
        elif content_str[i].count('gtad4xkn')==2:
            share=re.findall('공유 \d....',content_str[i])
            comment=re.findall('댓글 \d....',content_str[i]) 
            comment2=comment[0]
            sharecomment=comment2+', '+share[0]
            comment_gongyu_list.append(sharecomment)
    else:
        comment_gongyu_list.append('댓글 0개, 공유 0회')
comment_share_list=[]
for i in range(len(comment_gongyu_list)):
    comment_share=re.sub('[-=#/?:$<}s]', '', comment_gongyu_list[i])
    comment_share_list.append(comment_share)
results={'Date':time_list6,
        'Content':content_list2,
        'Num of Reaction':reaction_list,
        'Num of Comment&share':comment_share_list}
# driver.close()

In [4]:
df=pd.DataFrame(results)
df.to_excel('Facebook_Crwaling_KTO_Korean.xlsx',encoding='CP949')
df

,Date,Content,Num of Reaction,Num of Comment&share
0,11,대한민국 구석구석 · 11시간 · 눈이 내리고~ #ASMR 이 흐르면~이게 바로...,44,"댓글 2개, 공유 0회"
1,12월 14일 535,대한민국 구석구석 · ================================1...,50,"댓글 3개, 공유 0회"
2,12월 11일 400,대한민국 구석구석 · ================================1...,106,"댓글 2개, 공유 37회"
3,12월 10일 300,대한민국 구석구석 · ================================1...,65,"댓글 0개, 공유 1회"
4,12월 9일 400,대한민국 구석구석 · =======12월 9일 오후 4:00============...,106,"댓글 4개, 공유 14회"
...,...,...,...,...
1089,3 1 2018 2월 22일,서울 샥슈카 사진첩대한민국 구석구석님이 사진 3장 및 동영상 1개를 추가했습니다. ...,275,"댓글 139개, 공유 94회"
1090,2018 2월 21일,대한민국 구석구석 · ===========2018년 2월 21일==========...,352,"댓글 288개, 공유 436회"
1091,2018 2월 21일,대한민국 구석구석 · =======2018년 2월 21일==============...,30,"댓글 1개, 공유 0회"
1092,3 1 2018 2월 21일,광명동굴 사진첩대한민국 구석구석님이 사진 3장 및 동영상 1개를 추가했습니다. ·...,625,"댓글 417개, 공유 569회"
